In [1]:
# Install ipython-sql
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [2]:
# Load the SQL extension
%load_ext sql

# Create a SQLite database
%sql sqlite://

In [44]:
# Import necessary libraries and files
import sqlite3
import numpy as np
from google.colab import files
import pandas as pd
data=files.upload()

Saving sleepDay_merged.csv to sleepDay_merged.csv


In [4]:
# Read the file with Python
df = pd.read_csv("dailyActivity_merged.csv")
df['ActivityDate'].dtype

dtype('O')

In [5]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=7690e177cb348ed0f40cf42130e393e865ccf5e0a23aeda85e1137987b51d5a1
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [6]:
import sqlite3
import pandas as pd

# Connect to or create a new SQLite database
conn = sqlite3.connect('bellabeat.db')

# Read the CSV file into a DataFrame
df = pd.read_csv('dailyActivity_merged.csv')

data_types = {
    'Id': 'INTEGER',
    'ActivityDate': 'DATE',
    'TotalSteps': 'INTEGER',
    'TotalDistance': 'REAL',
    'TrackerDistance': 'REAL',
    'LoggedActivitiesDistance': 'REAL',
    'VeryActiveDistance': 'REAL',
    'ModeratelyActiveDistance': 'REAL',
    'LightActiveDistance': 'REAL',
    'SedentaryActiveDistance': 'REAL',
     'VeryActiveMinutes': 'INTEGER',
     'FairlyActiveMinutes': 'INTEGER',
     'LightlyActiveMinutes': 'INTEGER',
     'SedentaryMinutes': 'INTEGER',
      'Calories': 'INTEGER',
}

# Create a new table 'dailyActivity_merged' and insert data with specified data types
df.to_sql('dailyActivity_merged', conn, if_exists='replace', index=False, dtype=data_types)

conn.close()

In [7]:
pip install sqlalchemy ipython-sql

In [9]:
# Connect to the SQLite database
%sql sqlite:///bellabeat.db

In [10]:

# Check data types using SQL
%%sql
PRAGMA table_info(dailyActivity_merged);

   sqlite://
 * sqlite:///bellabeat.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Id,INTEGER,0,None,0
1,ActivityDate,DATE,0,None,0
2,TotalSteps,INTEGER,0,None,0
3,TotalDistance,REAL,0,None,0
4,TrackerDistance,REAL,0,None,0
5,LoggedActivitiesDistance,REAL,0,None,0
6,VeryActiveDistance,REAL,0,None,0
7,ModeratelyActiveDistance,REAL,0,None,0
8,LightActiveDistance,REAL,0,None,0
9,SedentaryActiveDistance,REAL,0,None,0


In [61]:
# Check the number of members/ID
%%sql
SELECT COUNT(DISTINCT Id) AS Distinct_Id_Count
FROM dailyActivity_merged;

   sqlite://
 * sqlite:///bellabeat.db
Done.


Distinct_Id_Count
33


In [60]:
# Get the number of members that have records almost everyday
%%sql
SELECT Id,
COUNT(Id) AS Total_Id
FROM dailyActivity_merged
GROUP BY Id
having Total_Id>=30

   sqlite://
 * sqlite:///bellabeat.db
Done.


Id,Total_Id
1503960366,31
1624580081,31
1644430081,30
1844505072,31
1927972279,31
2022484408,31
2026352035,31
2320127002,31
2873212765,31
3977333714,30


In [12]:
# Define active users and non-active users, and respective %
%%sql
WITH UserTypeCounts AS (
    SELECT
        Id,
        COUNT(Id) AS Total_Logged_Uses,
        CASE
            WHEN COUNT(Id) BETWEEN 25 AND 31 THEN 'Active User'
            WHEN COUNT(Id) BETWEEN 15 AND 24 THEN 'Moderate User'
            WHEN COUNT(Id) BETWEEN 0 AND 14 THEN 'Light User'
        END AS Fitbit_Usage_Type
    FROM dailyActivity_merged
    GROUP BY Id
),
TotalCounts AS (
    SELECT
        COUNT(Id) AS TotalUsers
    FROM UserTypeCounts
)
SELECT
    Fitbit_Usage_Type,
    COUNT(Id) AS UserCount,
    ROUND((COUNT(Id) * 100.0) / TotalUsers, 2) AS PercentageOfTotal
FROM UserTypeCounts, TotalCounts
GROUP BY Fitbit_Usage_Type
ORDER BY Fitbit_Usage_Type;

   sqlite://
 * sqlite:///bellabeat.db
Done.


Fitbit_Usage_Type,UserCount,PercentageOfTotal
Active User,29,87.88
Light User,1,3.03
Moderate User,3,9.09


In [13]:
# Statistical summary on steps, distance, calories, activity per ID account
%%sql
SELECT Id,
MIN(TotalSteps) AS Min_Total_Steps,
MAX(TotalSteps) AS Max_Total_Steps,
ROUND(AVG(TotalSteps)) AS Avg_Total_Stpes,
ROUND(MIN(TotalDistance)) AS Min_Total_Distance,
ROUND(MAX(TotalDistance)) AS Max_Total_Distance,
ROUND(AVG(TotalDistance)) AS Avg_Total_Distance,
MIN(Calories) AS Min_Total_Calories,
MAX(Calories) AS Max_Total_Calories,
ROUND(AVG(Calories)) AS Avg_Total_Calories,
MIN(VeryActiveMinutes) AS Min_Very_Active_Minutes,
MAX(VeryActiveMinutes) AS Max_Very_Active_Minutes,
ROUND(AVG(VeryActiveMinutes)) AS Avg_Very_Active_Minutes,
MIN(FairlyActiveMinutes) AS Min_Fairly_Active_Minutes,
MAX(FairlyActiveMinutes) AS Max_Fairly_Active_Minutes,
ROUND(AVG(FairlyActiveMinutes)) AS Avg_Fairly_Active_Minutes,
MIN(LightlyActiveMinutes) AS Min_Lightly_Active_Minutes,
MAX(LightlyActiveMinutes) AS Max_Lightly_Active_Minutes,
ROUND(AVG(LightlyActiveMinutes)) AS Avg_Lightly_Active_Minutes,
MIN(SedentaryMinutes) AS Min_Sedentary_Minutes,
MAX(SedentaryMinutes) AS Max_Sedentary_Minutes,
ROUND(AVG(SedentaryMinutes)) AS Avg_Sedentary_Minutes
From dailyActivity_merged
Group BY Id

   sqlite://
 * sqlite:///bellabeat.db
Done.


Id,Min_Total_Steps,Max_Total_Steps,Avg_Total_Stpes,Min_Total_Distance,Max_Total_Distance,Avg_Total_Distance,Min_Total_Calories,Max_Total_Calories,Avg_Total_Calories,Min_Very_Active_Minutes,Max_Very_Active_Minutes,Avg_Very_Active_Minutes,Min_Fairly_Active_Minutes,Max_Fairly_Active_Minutes,Avg_Fairly_Active_Minutes,Min_Lightly_Active_Minutes,Max_Lightly_Active_Minutes,Avg_Lightly_Active_Minutes,Min_Sedentary_Minutes,Max_Sedentary_Minutes,Avg_Sedentary_Minutes
1503960366,0,18134,12117.0,0.0,12.0,8.0,0,2159,1816.0,0,78,39.0,0,46,19.0,0,328,220.0,539,1440,848.0
1624580081,1510,36019,5744.0,1.0,28.0,4.0,1002,2690,1483.0,0,186,9.0,0,63,6.0,91,236,153.0,890,1349,1258.0
1644430081,1223,18213,7283.0,1.0,13.0,5.0,1276,3846,2811.0,0,51,10.0,0,94,21.0,38,402,178.0,682,1402,1162.0
1844505072,0,8054,2580.0,0.0,5.0,2.0,665,2130,1573.0,0,2,0.0,0,13,1.0,0,373,115.0,218,1440,1207.0
1927972279,0,3790,916.0,0.0,3.0,1.0,1383,2638,2173.0,0,16,1.0,0,9,1.0,0,192,39.0,734,1440,1317.0
2022484408,3292,18387,11371.0,2.0,13.0,8.0,1848,3158,2510.0,0,104,36.0,0,63,19.0,135,361,257.0,728,1305,1113.0
2026352035,254,12357,5567.0,0.0,8.0,3.0,1141,1926,1541.0,0,3,0.0,0,8,0.0,17,475,257.0,330,1062,689.0
2320127002,772,10725,4717.0,1.0,7.0,3.0,1125,2124,1724.0,0,20,1.0,0,11,3.0,40,335,198.0,830,1400,1220.0
2347167796,42,22244,9520.0,0.0,15.0,6.0,403,2670,2043.0,0,66,14.0,0,72,21.0,4,382,253.0,2,1011,687.0
2873212765,2524,9685,7556.0,2.0,7.0,5.0,1431,2241,1917.0,0,75,14.0,0,23,6.0,52,448,308.0,720,1302,1097.0


In [59]:
# Activity level per users
%%sql
SELECT Id,
round(avg(VeryActiveMinutes),1) AS Avg_Very_Active_Minutes,
round(avg(FairlyActiveMinutes),1) AS Avg_Fairly_Active_Minutes,
round(avg(LightlyActiveMinutes),1) AS Avg_Lightly_Active_Minutes,
round(avg(SedentaryMinutes),1) AS Avg_Sedentary_Minutes
From dailyActivity_merged
GROUP BY Id

   sqlite://
 * sqlite:///bellabeat.db
Done.


Id,Avg_Very_Active_Minutes,Avg_Fairly_Active_Minutes,Avg_Lightly_Active_Minutes,Avg_Sedentary_Minutes
1503960366,38.7,19.2,219.9,848.2
1624580081,8.7,5.8,153.5,1257.7
1644430081,9.6,21.4,178.5,1161.9
1844505072,0.1,1.3,115.5,1206.6
1927972279,1.3,0.8,38.6,1317.4
2022484408,36.3,19.4,257.5,1112.6
2026352035,0.1,0.3,256.6,689.4
2320127002,1.4,2.6,198.2,1220.1
2347167796,13.5,20.6,252.5,687.2
2873212765,14.1,6.1,308.0,1097.2


In [36]:
# Activities per weekday
%%sql
SELECT ActivityDate,
ROUND(avg(TotalSteps),2) AS AVG_TotalSteps,
ROUND(avg(TotalDistance),2) AS AVG_TotalDistance,
ROUND(avg(Calories),2) AS AVG_TotalCalories
FROM dailyActivity_merged
GROUP BY ActivityDate

   sqlite://
 * sqlite:///bellabeat.db
Done.


ActivityDate,AVG_TotalSteps,AVG_TotalDistance,AVG_TotalCalories
Friday,7448.23,5.31,2331.79
Monday,7780.87,5.55,2324.21
Saturday,8152.98,5.85,2354.97
Sunday,6933.23,5.03,2263.0
Thursday,7405.84,5.31,2199.57
Tuesday,8125.01,5.83,2356.01
Wednesday,7559.37,5.49,2302.62


In [32]:
# Check if the active minutes meet the recommended CDC threshold of 150 mins, and how many people
%%sql
SELECT CDC_Recommendations, COUNT(*) AS Recommendation_Count
FROM (
    SELECT Id,
           CASE
               WHEN ROUND(AVG(VeryActiveMinutes) + AVG(FairlyActiveMinutes) + avg(LightlyActiveMinutes))>= 150 THEN 'Meets CDC Recommendation'
               WHEN ROUND(AVG(VeryActiveMinutes) + AVG(FairlyActiveMinutes) + avg(LightlyActiveMinutes))< 150 THEN 'Does Not Meet CDC Recommendation'
           END AS CDC_Recommendations
    FROM dailyActivity_merged
    GROUP BY Id
) subquery
GROUP BY CDC_Recommendations;

   sqlite://
 * sqlite:///bellabeat.db
Done.


CDC_Recommendations,Recommendation_Count
Does Not Meet CDC Recommendation,6
Meets CDC Recommendation,27


In [54]:
# Import another table with sleep information
# Connect to or create a new SQLite database
conn = sqlite3.connect('bellabeat.db')

# Read the CSV file into a DataFrame
df2 = pd.read_csv("sleepDay_merged.csv")
# Create a new table 'dailyActivity_merged' and insert data with specified data types
df2.to_sql('sleepDay_merged', conn, if_exists='replace', index=False)

conn.close()

In [56]:
# Check if any relationship betwen steps and sleeps
%%sql
WITH temp_table AS (
  SELECT Id, ROUND(AVG(dailyActivity_merged.TotalSteps)) AS DailyAVGSteps
  FROM dailyActivity_merged
  GROUP BY Id
)

SELECT
  temp_table.Id,
  temp_table.DailyAVGSteps,
  round(AVG(sleepDay_merged.TotalTimeInBed)) AS AVGMinutesAsleep
FROM temp_table
INNER JOIN sleepDay_merged
ON sleepDay_merged.Id = temp_table.Id
GROUP BY temp_table.Id
ORDER BY temp_table.DailyAVGSteps DESC;

   sqlite://
 * sqlite:///bellabeat.db
Done.


Id,DailyAVGSteps,AVGMinutesAsleep
8053475328,14763.0,302.0
1503960366,12117.0,383.0
7007744171,11323.0,72.0
3977333714,10985.0,461.0
4388161847,10814.0,423.0
6962181067,9795.0,466.0
2347167796,9520.0,491.0
7086361926,9372.0,466.0
8378563200,8718.0,486.0
5553957443,8613.0,506.0
